In [15]:
import pandas as pd

df = pd.read_csv("../data_clean/metadata.csv")
df
# print(df['diagnosis'].unique())

,image_id,origin_dataset,lesion,diagnosis,localization,age,sex
0,ISIC_0024306.jpg,ISIC2020,benign,NV,trunk,45,male
1,ISIC_0024307.jpg,ISIC2020,benign,NV,lower extremity,50,male
2,ISIC_0024308.jpg,ISIC2020,benign,NV,trunk,55,female
3,ISIC_0024309.jpg,ISIC2020,benign,NV,trunk,40,male
4,ISIC_0024310.jpg,ISIC2020,malignant,MEL,chest,60,male
...,...,...,...,...,...,...,...
71710,ISIC_0073247.jpg,ISIC2020,malignant,BCC,head/neck,85,female
71711,ISIC_0073248.jpg,ISIC2020,benign,BKL,anterior torso,65,male
71712,ISIC_0073249.jpg,ISIC2020,malignant,MEL,lower extremity,70,male
71713,ISIC_0073251.jpg,ISIC2020,benign,NV,palms/soles,55,female


In [20]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.replace({'': pd.NA}, inplace=True)

# Detecta columna de dataset (proveniencia)
dataset_candidates = ['origin_dataset', 'dataset', 'source_dataset', 'origin', 'source']
dataset_col = next((c for c in dataset_candidates if c in df.columns), None)

miss = df.isna()
N = len(df)

# === Resumen global ===
overall = miss.sum().sort_values(ascending=False).to_frame('missing_count')
overall['missing_rate_%'] = (overall['missing_count'] / N * 100).round(2)

print(f"Total filas: {N}")
display(overall)

# === Desglose por dataset ===
if dataset_col:
    per_ds_counts = miss.groupby(df[dataset_col]).sum().sort_index()
    per_ds_rates = (per_ds_counts.div(df.groupby(dataset_col).size(), axis=0) * 100).round(2)

    print("\n# Faltantes por dataset (conteo):")
    display(per_ds_counts)

    print("\n# Faltantes por dataset (porcentaje %):")
    display(per_ds_rates)

    # Top contribuyentes a NaN por columna
    print("\n# Top datasets que más contribuyen a los NaNs por columna:")
    for col in df.columns:
        if overall.loc[col, 'missing_count'] == 0:
            continue
        s = per_ds_counts[col].sort_values(ascending=False)
        s = s[s > 0]
        if not s.empty:
            print(f"\nColumna: {col} (NaNs totales={overall.loc[col,'missing_count']})")
            print(s.head(5).to_string())

    # Filas con al menos un NaN por dataset
    any_missing_by_ds = miss.any(axis=1).groupby(df[dataset_col]).sum().sort_values(ascending=False)
    print("\n# Filas con AL MENOS un NaN por dataset (conteo):")
    display(any_missing_by_ds.to_frame('rows_with_any_NaN'))
else:
    print("\n(no se encontró columna de dataset; se muestran solo totales)")

# === Ejemplos de filas afectadas (muestra hasta 5 por columna) ===
example_cols = [c for c in ['lesion','diagnosis','localization','age','sex'] if c in df.columns]
if not example_cols:
    example_cols = list(df.columns)

id_like_cols = [c for c in ['image_id','filename','image','path','image_path','image_relpath'] if c in df.columns]
cols_show_base = (['image_id'] if 'image_id' in df.columns else []) + ([dataset_col] if dataset_col else []) + id_like_cols
cols_show_base = list(dict.fromkeys([c for c in cols_show_base if c]))  # dedup y limpia Nones

print("\n# Ejemplos de filas con NaN por columna:")
for col in example_cols:
    idx = df.index[miss[col]]
    if len(idx) == 0:
        continue
    print(f"\nColumna '{col}': mostrando hasta 5 ejemplos")
    cols_to_show = cols_show_base + [col]
    cols_to_show = list(dict.fromkeys([c for c in cols_to_show if c in df.columns]))
    display(df.loc[idx, cols_to_show].head(5))


Total filas: 71715


,missing_count,missing_rate_%
image_id,0,0.0
origin_dataset,0,0.0
lesion,0,0.0
diagnosis,0,0.0
localization,0,0.0
age,0,0.0
sex,0,0.0



# Faltantes por dataset (conteo):


,image_id,origin_dataset,lesion,diagnosis,localization,age,sex
origin_dataset,,,,,,,
ISIC2020,0,0,0,0,0,0,0
ITOBOS2024,0,0,0,0,0,0,0



# Faltantes por dataset (porcentaje %):


,image_id,origin_dataset,lesion,diagnosis,localization,age,sex
origin_dataset,,,,,,,
ISIC2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ITOBOS2024,0.0,0.0,0.0,0.0,0.0,0.0,0.0



# Top datasets que más contribuyen a los NaNs por columna:

# Filas con AL MENOS un NaN por dataset (conteo):


,rows_with_any_NaN
origin_dataset,
ISIC2020,0
ITOBOS2024,0



# Ejemplos de filas con NaN por columna:


In [17]:
mask = (df['origin_dataset'].eq('ISIC2019')) & (df['localization'].isna())
subset = df[mask]
subset

,image_id,origin_dataset,lesion,diagnosis,localization,age,sex


In [18]:
mask = (df['origin_dataset'].eq('MIL10K')) & (df['localization'].isna())
subset = df[mask]
subset

,image_id,origin_dataset,lesion,diagnosis,localization,age,sex


In [19]:
mask = (df['origin_dataset'].eq('ISIC2020')) & (df['localization'].isna())
subset = df[mask]
subset

,image_id,origin_dataset,lesion,diagnosis,localization,age,sex


## Comprobacion de que no hay imagenes duplicadas

In [5]:
import os, hashlib
from collections import defaultdict

DATA_DIR = r"..\data_clean"  # <- ajusta si procede
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}

# Límites de salida (puedes poner None para mostrar todo)
MAX_GROUPS_TO_SHOW = 50       # nº máximo de grupos duplicados a imprimir (None = todos)
MAX_PATHS_PER_GROUP = None    # nº máximo de rutas por grupo (None = todas)

def md5_of(path, chunk=1024*1024):
    h = hashlib.md5()
    with open(path, "rb") as f:
        while True:
            b = f.read(chunk)
            if not b: break
            h.update(b)
    return h.hexdigest()

# Indexa por hash
hash2paths = defaultdict(list)
total_imgs = 0
for root, _, fns in os.walk(DATA_DIR):
    for f in fns:
        if os.path.splitext(f.lower())[1] in IMG_EXTS:
            p = os.path.join(root, f)
            total_imgs += 1
            try:
                h = md5_of(p)
                hash2paths[h].append(p)
            except Exception as e:
                print("ERROR leyendo:", p, e)

# Detecta duplicados
dups = {h: ps for h, ps in hash2paths.items() if len(ps) > 1}

# Resumen
num_unique_hashes = len(hash2paths)
num_dup_groups = len(dups)
dup_files_extras = sum(len(ps) - 1 for ps in dups.values())  # archivos que "sobran"

print("=== Resumen de duplicados por contenido (MD5) ===")
print(f"Imágenes escaneadas:        {total_imgs}")
print(f"Hashes únicos:              {num_unique_hashes}")
print(f"Grupos con duplicados:      {num_dup_groups}")
print(f"Ficheros duplicados extra:  {dup_files_extras}")
print("-" * 50)

# Listado de duplicados (si los hay)
if num_dup_groups == 0:
    print("✅ No hay duplicados por contenido (MD5).")
else:
    print("⚠️ Se han encontrado duplicados. Detalle por grupo (ordenado por tamaño):")
    sorted_groups = sorted(dups.items(), key=lambda kv: -len(kv[1]))
    if isinstance(MAX_GROUPS_TO_SHOW, int):
        sorted_groups = sorted_groups[:MAX_GROUPS_TO_SHOW]

    for idx, (h, ps) in enumerate(sorted_groups, 1):
        print(f"\n[{idx}] MD5: {h}  |  count={len(ps)}")
        paths_to_show = ps if MAX_PATHS_PER_GROUP is None else ps[:MAX_PATHS_PER_GROUP]
        for p in paths_to_show:
            print("   -", p)
        if MAX_PATHS_PER_GROUP is not None and len(ps) > MAX_PATHS_PER_GROUP:
            print(f"   ... (+{len(ps) - MAX_PATHS_PER_GROUP} rutas más)")
    
    if isinstance(MAX_GROUPS_TO_SHOW, int) and num_dup_groups > MAX_GROUPS_TO_SHOW:
        print(f"\n... (+{num_dup_groups - MAX_GROUPS_TO_SHOW} grupos más no mostrados)")


=== Resumen de duplicados por contenido (MD5) ===
Imágenes escaneadas:        71715
Hashes únicos:              71715
Grupos con duplicados:      0
Ficheros duplicados extra:  0
--------------------------------------------------
✅ No hay duplicados por contenido (MD5).


image_id             0
origin_dataset       0
lesion            1511
diagnosis         1511
localization      3029
age               2031
sex               3691
dtype: int64